In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
tables = ['employee', 'project']
for table in tables:
    display_table(table)

+-------------+--------+------------------+
| employee_id |  name  | experience_years |
+-------------+--------+------------------+
|      1      | Khaled |        3         |
|      2      |  Ali   |        2         |
|      3      |  John  |        1         |
|      4      |  Doe   |        2         |
+-------------+--------+------------------+
+------------+-------------+
| project_id | employee_id |
+------------+-------------+
|     1      |      1      |
|     1      |      2      |
|     1      |      3      |
|     2      |      1      |
|     2      |      4      |
+------------+-------------+


### Write an SQL query that reports the average experience years of all the employees for each project, rounded to 2 digits.
```
Output: 
+-------------+---------------+
| project_id  | average_years |
+-------------+---------------+
| 1           | 2.00          |
| 2           | 2.50          |
+-------------+---------------+
Explanation: The average experience years for the first project is (3 + 2 + 1) / 3 = 2.00 and for the second project is (3 + 2) / 2 = 2.50
```

In [3]:
%%sql
 
SELECT p.project_id, ROUND(AVG(experience_years), 2) AS average_years
FROM employee e 
JOIN project p ON e.employee_id = p.employee_id
GROUP BY project_id

project_id,average_years
1,2.00
2,2.50


# Using Pandas

In [4]:
import pandas as pd 

In [5]:
employee_query = %sql SELECT * FROM employee # type: ignore 
project_query = %sql SELECT * FROM project # type: ignore 

employee_df = employee_query.DataFrame()
project_df = project_query.DataFrame()

display(employee_df, project_df)

,employee_id,name,experience_years
0,1,Khaled,3
1,2,Ali,2
2,3,John,1
3,4,Doe,2


,project_id,employee_id
0,1,1
1,1,2
2,1,3
3,2,1
4,2,4


In [6]:
merged_df = employee_df.merge(project_df, on='employee_id', how='outer')
merged_df

,employee_id,name,experience_years,project_id
0,1,Khaled,3,1
1,1,Khaled,3,2
2,2,Ali,2,1
3,3,John,1,1
4,4,Doe,2,2


In [7]:
merged_df.groupby('project_id').agg({'experience_years':'mean'})

,experience_years
project_id,
1,2.0
2,2.5


In [8]:
merged_df.groupby('project_id') \
    .agg({'experience_years':'mean'}) \
    .round(2) \
    .rename(columns={'experience_years': 'average_years'})

,average_years
project_id,
1,2.0
2,2.5
